In [1]:
import nflfastpy as npy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager

### Add Font - Poppins from Google Fonts [(Link)](https://fonts.google.com/specimen/Poppins?query=poppins)

In [2]:
font_paths = [
    "../../fonts/Poppins-Regular.ttf",
    "../../fonts/Poppins-Medium.ttf",
    "../../fonts/Poppins-SemiBold.ttf",
    "../../fonts/Poppins-Bold.ttf"
]

for path in font_paths:
    font_manager.fontManager.addfont(path)
    font_manager.FontProperties(fname=path)

In [40]:
df_2021 = npy.load_pbp_data(2021)

In [11]:
for column in df_2021.columns:
    print(column)

play_id
game_id
old_game_id
home_team
away_team
season_type
week
posteam
posteam_type
defteam
side_of_field
yardline_100
game_date
quarter_seconds_remaining
half_seconds_remaining
game_seconds_remaining
game_half
quarter_end
drive
sp
qtr
down
goal_to_go
time
yrdln
ydstogo
ydsnet
desc
play_type
yards_gained
shotgun
no_huddle
qb_dropback
qb_kneel
qb_spike
qb_scramble
pass_length
pass_location
air_yards
yards_after_catch
run_location
run_gap
field_goal_result
kick_distance
extra_point_result
two_point_conv_result
home_timeouts_remaining
away_timeouts_remaining
timeout
timeout_team
td_team
td_player_name
td_player_id
posteam_timeouts_remaining
defteam_timeouts_remaining
total_home_score
total_away_score
posteam_score
defteam_score
score_differential
posteam_score_post
defteam_score_post
score_differential_post
no_score_prob
opp_fg_prob
opp_safety_prob
opp_td_prob
fg_prob
safety_prob
td_prob
extra_point_prob
two_point_conversion_prob
ep
epa
total_home_epa
total_away_epa
total_home_rush_epa


### Count sacks against passing yards

In [41]:
df_2021["passing_yards"] = df_2021.apply(lambda row: row["passing_yards"] if not row["sack"] else row["yards_gained"], axis=1)
df_2021["passing_yards"] = df_2021["passing_yards"].fillna(0)
df_2021["rushing_yards"] = df_2021["rushing_yards"].fillna(0)

In [108]:
df_2021_sub = df_2021[df_2021["two_point_attempt"] == 0].copy()

In [109]:
df_2021_sub["total_yards"] = df_2021_sub["passing_yards"] + df_2021_sub["rushing_yards"]

In [116]:
df_off = df_2021_sub.groupby("posteam", as_index=False).agg({"yards_gained": "sum", "passing_yards": "sum", "rushing_yards": "sum"})
df_off["pass_and_rush_yds"] = df_off["passing_yards"] + df_off["rushing_yards"]

# Correction for aborted plays
df_off["rushing_yards"] = df_off.apply(lambda row: row["rushing_yards"] + (row["yards_gained"] - row["pass_and_rush_yds"]), axis=1)

# Calculate yards per game
df_off["yds_per_game"] = (df_off["yards_gained"] / 15).round(1)
df_off["pass_yds_per_game"] = (df_off["passing_yards"] / 15).round(1)
df_off["rush_yds_per_game"] = (df_off["rushing_yards"] / 15).round(1)

# Drop pass_and_rush_yds column
df_off = df_off.drop(columns=["pass_and_rush_yds"])

df_off

,posteam,yards_gained,passing_yards,rushing_yards,yds_per_game,pass_yds_per_game,rush_yds_per_game
0,ARI,5648.0,3817.0,1831.0,376.5,254.5,122.1
1,ATL,4642.0,3349.0,1293.0,309.5,223.3,86.2
2,BAL,5732.0,3667.0,2065.0,382.1,244.5,137.7
3,BUF,5718.0,3912.0,1806.0,381.2,260.8,120.4
4,CAR,4586.0,2942.0,1644.0,305.7,196.1,109.6
5,CHI,4620.0,2779.0,1841.0,308.0,185.3,122.7
6,CIN,5488.0,3885.0,1603.0,365.9,259.0,106.9
7,CLE,5183.0,3010.0,2173.0,345.5,200.7,144.9
8,DAL,6143.0,4240.0,1903.0,409.5,282.7,126.9
9,DEN,4935.0,3184.0,1751.0,329.0,212.3,116.7
